# Table of Contents
01. Importing Libraries
02. Importing Data
03. Security Implications
04. Regional Segmentation
05. Excluding Low-Activity Customers
06. Customer Profiling
07. Aggregating Data from Customer Profiles
08. Exporting Data

# 01. Importing Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

# 02. Importing Data

In [2]:
# Create path
path = r'C:\Users\erj03\OneDrive\Documents\Achievment 4- Instacart Basket Analysis'

In [3]:
# Import data set
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'new_dataframe.pkl'))

In [4]:
# View data frame
ords_prods_merge.head()

,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,First Name,Surname,Gender,STATE,Age,date_joined,n_dependants,marital_status,income,merge_indicator
0,2539329,1,1,2,8,NaN,196,1,0,both,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,both,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,both,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,both,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,both,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [5]:
# Check dimensions of dataframe
ords_prods_merge.shape

(32404859, 33)

# 03. Security Implications

Our data set contains first and last names of customers as well as the states they live in. Usually this would be of concern, but as addressed in our project brief the customers in this specific data set have been fabricated. If we were dealing with legitimate customers, we would reach out to whomever was in charge of data security to determine if the names should be kept in the data set or removed. This would not have a great affect on our data because each customer can be indentifid by their unique user ID.

# 04. Regional Segmentation

Stakeholders would like to compare spending behavior across different regions of the country. To do this, we will use the state column to group the data by region accroding to the provided source (https://simple.wikipedia.org/wiki/List_of_regions_of_the_United_States). We will then determine whether there’s a difference in spending habits between the different U.S. regions.

## Create Region Column

In [6]:
# Define regions
northeast_states = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
midwest_states = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
south_states = ['Delaware', 'Alabama', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'ALabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
west_states = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [7]:
# Define northeast with loc function
ords_prods_merge.loc[ords_prods_merge['STATE'].isin(northeast_states), 'region'] = 'Northeast'

In [8]:
# Define midwest with loc function
ords_prods_merge.loc[ords_prods_merge['STATE'].isin(midwest_states), 'region'] = 'Midwest'

In [9]:
# Define south with loc function
ords_prods_merge.loc[ords_prods_merge['STATE'].isin(south_states), 'region'] = 'South'

In [10]:
# Define west with loc function
ords_prods_merge.loc[ords_prods_merge['STATE'].isin(west_states), 'region'] = 'West'

In [11]:
ords_prods_merge['region'].value_counts()

region
South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: count, dtype: int64

## Determine Spending Behavior by Region

In [12]:
# Create crosstab with region and spending_flag columns
crosstab = pd.crosstab(ords_prods_merge['region'], ords_prods_merge['spending_flag'], dropna = False)

In [13]:
display(crosstab)

spending_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


In [14]:
# Look at normalization of each region
crosstab_norm = pd.crosstab(ords_prods_merge['region'], ords_prods_merge['spending_flag'], dropna = False, normalize = 'index')

In [15]:
display(crosstab_norm)

spending_flag,High spender,Low spender
region,,
Midwest,0.020530,0.979470
Northeast,0.018911,0.981089
South,0.019430,0.980570
West,0.019336,0.980664


In [16]:
# Calculate by percentage
crosstab_norm * 100

spending_flag,High spender,Low spender
region,,
Midwest,2.053025,97.946975
Northeast,1.891141,98.108859
South,1.943043,98.056957
West,1.933627,98.066373


In [17]:
# Look at mean price for each region
ords_prods_merge.groupby('region').agg({'prices':['mean']})

,prices
,mean
region,
Midwest,7.792584
Northeast,7.782409
South,7.794918
West,7.790355


# 05. Excluding Low-Activity Customers

Low-activity customers that do not create much revenue are not of interest to our stakeholders. We will exclude customers with less than 5 orders from our data.

In [18]:
# Use loc function to create low activity flag
ords_prods_merge.loc[ords_prods_merge['max_order'] < 5, 'user_activity'] = 'low activity'

In [19]:
# Use loc function to create high activity flag
ords_prods_merge.loc[ords_prods_merge['max_order'] >= 5, 'user_activity'] = 'high activity'

In [20]:
# View updated dataframe
ords_prods_merge.head()

,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,Gender,STATE,Age,date_joined,n_dependants,marital_status,income,merge_indicator,region,user_activity
0,2539329,1,1,2,8,NaN,196,1,0,both,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,high activity
1,2398795,1,2,3,7,15.0,196,1,1,both,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,high activity
2,473747,1,3,3,12,21.0,196,1,1,both,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,high activity
3,2254736,1,4,4,7,29.0,196,1,1,both,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,high activity
4,431534,1,5,4,15,28.0,196,1,1,both,...,Female,Alabama,31,2/17/2019,3,married,40423,both,South,high activity


In [21]:
# Check value counts for both flags
ords_prods_merge['user_activity'].value_counts()

user_activity
high activity    30964564
low activity      1440295
Name: count, dtype: int64

In [22]:
# Create subset with only high activity customers
high_activity_customers = ords_prods_merge[ords_prods_merge['user_activity'] == 'high activity']

In [23]:
# Make sure dimensions of subset match up with the count of high activity users
high_activity_customers.shape

(30964564, 35)

In [24]:
# Export subset as pickle file 
high_activity_customers.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'highactivitycustomers.pkl'))

# 06. Customer Profiling

The marketing and business strategy units at Instacart want to create more-relevant marketing strategies for different products and are, thus, curious about customer profiling in their database. Create a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents. You might also use the “orders_day_of_week” and “order_hour_of_day” columns if you can think of a way they would impact customer profiles. (Hint: As an example, try thinking of what characteristics would lead you to the profile “Single adult” or “Young parent.”)

## Income vs Age

I will start by classifying each user as young adult (<30), adult (30-64), or senior (65+). I will also classify by low income, middle income, and high income. Income classifcations are according to https://www.weforum.org/agenda/2022/07/household-income-distribution-wealth-inequality-united-states/#:~:text=Here%E2%80%99s%20a%20look%20at%20the%20economic%20class%20breakdowns,2%20Middle%20class%3A%20%2452%2C000-%24156%2C000%203%20Lower%20class%3A%20%3C%2452%2C000

In [25]:
# Define low income young adult
ords_prods_merge.loc[(ords_prods_merge['income'] < 52000) & (ords_prods_merge['Age'] < 30), 'age_income_loc'] = 'low income young adult'

In [26]:
# Define middle income young adult
ords_prods_merge.loc[(ords_prods_merge['income'] >= 52000) & (ords_prods_merge['income'] <= 156000) & (ords_prods_merge['Age'] < 30), 'age_income_loc'] = 'middle income young adult'

In [27]:
# Define high income young adult
ords_prods_merge.loc[(ords_prods_merge['income'] >156000) &  (ords_prods_merge['Age'] < 30), 'age_income_loc'] = 'high income young adult'

In [28]:
# Define low income adult
ords_prods_merge.loc[(ords_prods_merge['income'] < 52000) & (ords_prods_merge['Age'] >= 30) & (ords_prods_merge['Age'] < 65), 'age_income_loc'] = 'low income adult'

In [29]:
# Define middle income adult
ords_prods_merge.loc[(ords_prods_merge['income'] >= 52000) & (ords_prods_merge['income'] <= 156000) & (ords_prods_merge['Age'] >= 30) & (ords_prods_merge['Age'] < 65), 'age_income_loc'] = 'middle income adult'

In [30]:
# Define high income adult
ords_prods_merge.loc[(ords_prods_merge['income'] > 156000) & (ords_prods_merge['Age'] >= 30) & (ords_prods_merge['Age'] < 65), 'age_income_loc'] = 'high income adult'

In [31]:
# Define low income senior
ords_prods_merge.loc[(ords_prods_merge['income'] < 52000) & (ords_prods_merge['Age'] >= 65), 'age_income_loc'] = 'low income senior'

In [32]:
# Define middle income senior
ords_prods_merge.loc[(ords_prods_merge['income'] >= 52000) & (ords_prods_merge['income'] <= 156000) & (ords_prods_merge['Age'] >= 65), 'age_income_loc'] = 'middle income senior'

In [33]:
# Define high income senior
ords_prods_merge.loc[(ords_prods_merge['income'] > 156000) & (ords_prods_merge['Age'] >= 65), 'age_income_loc'] = 'high income senior'

In [34]:
# View updated dataframe
ords_prods_merge.head()

,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,STATE,Age,date_joined,n_dependants,marital_status,income,merge_indicator,region,user_activity,age_income_loc
0,2539329,1,1,2,8,NaN,196,1,0,both,...,Alabama,31,2/17/2019,3,married,40423,both,South,high activity,low income adult
1,2398795,1,2,3,7,15.0,196,1,1,both,...,Alabama,31,2/17/2019,3,married,40423,both,South,high activity,low income adult
2,473747,1,3,3,12,21.0,196,1,1,both,...,Alabama,31,2/17/2019,3,married,40423,both,South,high activity,low income adult
3,2254736,1,4,4,7,29.0,196,1,1,both,...,Alabama,31,2/17/2019,3,married,40423,both,South,high activity,low income adult
4,431534,1,5,4,15,28.0,196,1,1,both,...,Alabama,31,2/17/2019,3,married,40423,both,South,high activity,low income adult


In [35]:
# Check value counts of new column
distribution = ords_prods_merge['age_income_loc'].value_counts()

In [36]:
display(distribution)

age_income_loc
middle income adult          13949461
middle income senior          6627494
middle income young adult     4996840
low income adult              2059960
high income adult             1729913
high income senior            1169364
low income young adult        1041267
low income senior              778226
high income young adult         52334
Name: count, dtype: int64

In [37]:
# Calculate normalization of age vs income groups
normalized = distribution / len(ords_prods_merge)

In [38]:
display(normalized)

age_income_loc
middle income adult          0.430474
middle income senior         0.204522
middle income young adult    0.154200
low income adult             0.063569
high income adult            0.053384
high income senior           0.036086
low income young adult       0.032133
low income senior            0.024016
high income young adult      0.001615
Name: count, dtype: float64

## Family Status

I will flag each customer as either childfree or parent based on if they have dependants and if they are married/unmarried.

In [40]:
# Create flag for family status of customer
ords_prods_merge.loc[(ords_prods_merge['n_dependants'] == 0) & (ords_prods_merge['marital_status'] == 'married'), 'family_status'] = 'married, childfree'

In [41]:
condition = (ords_prods_merge['n_dependants'] == 0) & (ords_prods_merge['marital_status'].isin(['divorced/widowed', 'single', 'living with parents and siblings']))
ords_prods_merge.loc[condition, 'family_status'] = 'unmarried, childfree'

In [42]:
condition = (ords_prods_merge['n_dependants'] != 0) & (ords_prods_merge['marital_status'].isin(['divorced/widowed', 'single', 'living with parents and siblings']))
ords_prods_merge.loc[condition, 'family_status'] = 'unmarried, parent'

In [43]:
ords_prods_merge.loc[(ords_prods_merge['n_dependants'] != 0) & (ords_prods_merge['marital_status'] == 'married'), 'family_status'] = 'married, parent'

In [44]:
# View updated dataframe
ords_prods_merge.head()

,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,Age,date_joined,n_dependants,marital_status,income,merge_indicator,region,user_activity,age_income_loc,family_status
0,2539329,1,1,2,8,NaN,196,1,0,both,...,31,2/17/2019,3,married,40423,both,South,high activity,low income adult,"married, parent"
1,2398795,1,2,3,7,15.0,196,1,1,both,...,31,2/17/2019,3,married,40423,both,South,high activity,low income adult,"married, parent"
2,473747,1,3,3,12,21.0,196,1,1,both,...,31,2/17/2019,3,married,40423,both,South,high activity,low income adult,"married, parent"
3,2254736,1,4,4,7,29.0,196,1,1,both,...,31,2/17/2019,3,married,40423,both,South,high activity,low income adult,"married, parent"
4,431534,1,5,4,15,28.0,196,1,1,both,...,31,2/17/2019,3,married,40423,both,South,high activity,low income adult,"married, parent"


In [45]:
# Check length of new column
len(ords_prods_merge['family_status'])

32404859

In [46]:
# Calculate distribution of new column
fam_distribution = ords_prods_merge['family_status'].value_counts()

In [47]:
display(fam_distribution)

family_status
married, parent         22756753
unmarried, childfree     8097503
unmarried, parent        1550603
Name: count, dtype: int64

In [48]:
# Calculate normalization of dependant status
fam_dis_normal = fam_distribution / len(ords_prods_merge)

In [49]:
display(fam_dis_normal)

family_status
married, parent         0.702264
unmarried, childfree    0.249885
unmarried, parent       0.047851
Name: count, dtype: float64

# 07. Aggregating Data from Customer Profiles

## Age vs Income

In [50]:
# Determine usage frequency of each age vs. income group by aggregating days_since_prior_order column
ords_prods_merge.groupby('age_income_loc')['days_since_prior_order'].agg(['mean', 'min', 'max'])

,mean,min,max
age_income_loc,,,
high income adult,10.921269,0.0,30.0
high income senior,10.935486,0.0,30.0
high income young adult,10.233835,0.0,30.0
low income adult,11.599392,0.0,30.0
low income senior,11.497626,0.0,30.0
low income young adult,11.563914,0.0,30.0
middle income adult,11.066034,0.0,30.0
middle income senior,11.055276,0.0,30.0
middle income young adult,11.029558,0.0,30.0


In [51]:
# Caculate mean, min, and max of expenditure for age vs income groups
ords_prods_merge.groupby('age_income_loc')['prices'].agg(['mean', 'min', 'max'])

,mean,min,max
age_income_loc,,,
high income adult,7.940265,1.0,25.0
high income senior,7.954955,1.0,25.0
high income young adult,7.803276,1.0,25.0
low income adult,6.720412,1.0,25.0
low income senior,6.500395,1.0,25.0
low income young adult,6.962268,1.0,25.0
middle income adult,7.930591,1.0,25.0
middle income senior,7.920826,1.0,25.0
middle income young adult,7.954097,1.0,25.0


## Family Status

In [52]:
# Determine usage frequency of each family status group by aggregating days_since_prior_order column
ords_prods_merge.groupby('family_status')['days_since_prior_order'].agg(['mean', 'min', 'max'])

,mean,min,max
family_status,,,
"married, parent",11.116175,0.0,30.0
"unmarried, childfree",11.080805,0.0,30.0
"unmarried, parent",11.048170,0.0,30.0


In [53]:
# Caculate mean, min, and max of expenditure for family status groups
ords_prods_merge.groupby('family_status')['prices'].agg(['mean', 'min', 'max'])

,mean,min,max
family_status,,,
"married, parent",7.790695,1.0,25.0
"unmarried, childfree",7.789975,1.0,25.0
"unmarried, parent",7.800712,1.0,25.0


# 08. Exporting Data

In [54]:
# Export dataframe as pickle file
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'final2_dataframe.pkl'))

In [55]:
# Export age vs income chart
bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'age_income.png'))